In [ ]:
! git clone  https://github.com/Jakobovski/free-spoken-digit-dataset 

Cloning into 'free-spoken-digit-dataset'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 3166 (delta 3), reused 8 (delta 3), pack-reused 3157
Receiving objects: 100% (3166/3166), 23.92 MiB | 23.55 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [22]:
from IPython.display import display, Audio
import librosa 
import numpy as np

In [ ]:
display(Audio(filename="recordings/7_jackson_37.wav", rate=1))

In [ ]:
! ls /content/free-spoken-digit-dataset/recordings/

In [11]:
file_path = "/content/free-spoken-digit-dataset/recordings/5_nicolas_14.wav"


In [13]:
samples, sample_rate = librosa.load(file_path)
for ts in [0.75, 1, 1.25]:
  for ps in [-1, 0, +1]:
    samples_new = librosa.effects.time_stretch(samples, rate=ts)
    y_new = librosa.effects.pitch_shift(samples_new, sample_rate, n_steps=ps)

In [23]:
max_length = 1.5 # Max length in seconds
samples, sample_rate = librosa.load(file_path)
short_samples = librosa.util.fix_length(samples, sample_rate*max_length)

melSpectrum = librosa.feature.melspectrogram(short_samples.astype(np.float16), sr=sample_rate, n_mels=128)

logMelSpectrogram = librosa.power_to_db(melSpectrum, ref=np.max)

TypeError: ignored

In [ ]:
import torch.nn as nn


#Pytorch Network Definition
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.fc1 = nn.Linear(3072, 128)
    self.fc2 = nn.Linear(128, 128)
    self.fc3 = nn.linear(128, 10)

  def forward(self, x):
    x = x.view((-1, 3072))  #---converts 2D data to 1D
    h = self.fc1(x)
    h = torch.relu(h)

    h = self.fc2(h)
    h = torch.relu(h)

    h = self.fc3(h)
    out = torch.log_softmax(h, dim=1)

    return out


In [ ]:
#Neural Network training in Pytorch
model = Model()
model.train()

if use_cuda:
  model.cuda()

optimizer = optim.Adam(model.parameters(), lr=0.01)
n_epoch = 40

for epoch in range(n_epoch):
  for data, target in train_loader:

    #Get Samples
    if use_cuda:
      data, target = data.cuda(), target.cuda()

    #Clear Gradient 
    optimizer.zero_grad()
    
    #Forward Propagation
    y_pred = model(data)

    #Error Computation
    loss = torch.cross_entropy(y_pred, target)

    #Back propagation
    loss.backward()

    #parameter Update
    optimizer.step()

# Modify the network to include some of the regularization techniques and activation functions such as,
# batch normalization, dropout, and ReLUs 

class Model2(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.fc1 = nn.Linear(3072, 128)
    self.bc1 = nn.BatchNorm1d(128)

    self.fc2 = nn.Linear(128, 128)
    self.bc2 = nn.BatchNorm1d(128)

    self.fc3 = nn.Linear(128, 10)
  
  def forward(self, x):
    x = x.view((-1, 3072))
    h = self.fc1(x)
    h = self.bc1(h)
    h = self.relu(h)
    h = F.dropout(h, p=0.5, training = self.training) # Disabled during evaluation

    h = self.fc2(h)
    h = self.bc2(h)
    h = self.relu(h)
    h = F.dropout(h, p=0.2, training = self.training) # Disabled during evaluation

    h = self.fc3(h)
    out = torch.log_softmax(h, dim=1)

    return out

In [ ]:
# UNSUPERVISED LEARNING

import torch.nn.functional as F

#Pytorch Network Definition 

class autoencoder(nn.Module):
  def __init__(self):
    super(autoencoder, self).__init__()
    self.e_fc1 = nn.Linear(3072, 512)
    self.e_fc2 = nn.Linear(512, 128)
    self.e_fc3 = nn.Linear(128, 64)
    self.e_fc4 = nn.Linear(64, 64)

    self.d_fc1 = nn.Linear(64, 64)
    self.d_fc2 = nn.Linear(128, 64)
    self.d_fc3 = nn.Linear(512, 128)
    self.d_fc4 = nn.Linear(3072, 512)

  
  def forward(self, x):
    #Encoder
    h = F.relu(self.e_fc1(x))
    h = F.relu(self.e_fc2(h))
    h = F.relu(self.e_fc3(h))
    h = self.e_fc4(h)

    #Decoder
    h = F.relu(self.d_fc1(h))
    h = F.relu(self.d_fc2(h))
    h = F.relu(self.d_fc3(h))
    h = self.d_fc4(h)
    out = F.tanh(h)

    return out
    

In [ ]:
import torch.optim as optim

model = autoencoder()
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weigth_decay = 1e-5)

for epoch in range (n_epoch):
  for data, _ in train_loader:
    #Get Samples

    input = data.view(-1, 3072) # we wil reuse the formatted input as our target

    #Forward Propagation 
    output = model(input)

    #Error Computation 
    loss = F.mse_loss(output, input)

    #Clear Gradient 
    optimizer.zero_grad()

    #Backpropagation 
    loss.backward()

    #Parameter Update 
    optimizer.step()




In [ ]:
# training with RBM code

class RBM(nn.Module):
    def __init__(self, n_vis=3072, n_hin=128, k=5):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(n_hin, n_vis,)* 1e-2)
        self.v_bias = nn.Parameter(torch.zeros(n_vis))
        self.h_bias = nn.Parameter(torch.zeros(n_hin))
        self.k = k

    def sample_from_p(self, p):
        return F.relu(torch.sign(P - Variable(torch.rand(p.size()))))

    def v_to_h(self, v):
        p_h = F.sigmoid(F.linear(v, self.W, self.h_bias))
        sample_h = self.sample_from_p(p_h)
        return sample_h, p_h

    def h_to_v(self, h):
        p_v = F.sigmoid(F.linear(h, self.W.t(), self.v_bias))
        sample_v = self.sample_from_p(p_v)
        return p_v, sample_v

    def forward(self, v):
        pre_h1, h1 = self.v_to_h(v)

        h_ = h1
        for _ in range (self.k):
        pre_v_, v_ = self.h_to_v(h_)
        pre_h_, h_ = self.v_to_h(v_)

        return v, v_

    def free_energy(self, v):
        vbias_term = v.mv(self.v_bias)
        wx_b = F.linear(v, self.W, self.h_bias)
        hidden_term = wx_b.exp().add(1).log().sum(1)
        return (- hidden_term - vbias_term).mean()


#Let's train the model with Adam

rbm = RBM(n_vis=3072, n_hin=128, k = 1)

train_op = optim.Adam(rbm.parameters(), 0.01)

for epoch in range (epochs)
    loss_ = []

    for _, (data, target) in enumerate (train_loader):
        data = Variable(data.view(-1, 3072))
        sample_data = data.bernoulli()

        v, v1 = rbm(sample_data)
        loss = rbm.free_energy(v) - rbm.free_energy(v1)
        loss_.append(loss.data[0])
        train_op.zero_grad()
        loss.backward()
        train_op.step()
# After training our RBM features, we can create a logistic regression classifier to
# classify our examples based on the unsupervised features we have learned'"



from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(train_features, train_labels)
predictions = clf.predict(test_features)